In [45]:
##connection 
import mysql.connector as conn 
username = 'root'
password = 'Newvision@143'
database = ''
try :
    connection = conn.connect(username =username,password=password,database =database)
    print("Successfully Connected")
except conn.Error as er :
    if er.errno == 1045:
        print(f'Access denied for user {username}')
    elif er.errno == 1049:
        print(f"Unknown database {database}")
    else :
        print(er)
     

Successfully Connected


In [46]:
cursor = connection.cursor()

In [48]:
##use database
use_database ='use litte_lemon_store '
cursor.execute(use_database)

Task 1

In [27]:
from mysql.connector.pooling import MySQLConnectionPool
dbconfig = {
    "database": "litte_lemon_store",
    "user": "root",
    "password": "Newvision@143"
}

try:
    
    pool_b = MySQLConnectionPool(pool_name="pool_b", pool_size=2, **dbconfig)
    print("Connection pool 'pool_b' created successfully.")

except conn.Error as e:
    print("Error:", e)


Connection pool 'pool_b' created successfully.


Task 2

In [19]:


try:
    # Establish a connection pool [pool_name = pool_b] with two connections
    pool_b = MySQLConnectionPool(pool_name="pool_b", pool_size=4, **dbconfig)

    # Connect the first guest
    connection1 = pool_b.get_connection()
    cursor1 = connection1.cursor()

    # Insert data for Guest 1
    guest1_data = {
        "table_number": 8,
        "first_name": "Anees",
        "last_name": "Java",
        "booking_time": "18:00",
        "employee_id": 6
    }
    insert_query1 = "INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES (%(table_number)s, %(first_name)s, %(last_name)s, %(booking_time)s, %(employee_id)s)"
    cursor1.execute(insert_query1, guest1_data)
    connection1.commit()

    # Connect the second guest
    connection2 = pool_b.get_connection()
    cursor2 = connection2.cursor()

    # Insert data for Guest 2
    guest2_data = {
        "table_number": 5,
        "first_name": "Bald",
        "last_name": "Vin",
        "booking_time": "19:00",
        "employee_id": 6
    }
    insert_query2 = "INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES (%(table_number)s, %(first_name)s, %(last_name)s, %(booking_time)s, %(employee_id)s)"
    cursor2.execute(insert_query2, guest2_data)
    connection2.commit()

    # Connect the third guest
    try:
        connection3 = pool_b.get_connection()
        cursor3 = connection3.cursor()

        # Insert data for Guest 3
        guest3_data = {
            "table_number": 12,
            "first_name": "Jay",
            "last_name": "Kon",
            "booking_time": "19:30",
            "employee_id": 6
        }
        insert_query3 = "INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES (%(table_number)s, %(first_name)s, %(last_name)s, %(booking_time)s, %(employee_id)s)"
        cursor3.execute(insert_query3, guest3_data)
        connection3.commit()
    
    except Exception as e:
        print("Error:", e)
        print("Could not connect the third guest.")

    # Task 2: Data inserted successfully for all three guests

except conn.Error as e:
    print("Error:", e)

finally:
    # Return the connections back to the pool
    if connection1.is_connected():
        cursor1.close()
        connection1.close()
        print("Connection 1 closed.")

    if connection2.is_connected():
        cursor2.close()
        connection2.close()
        print("Connection 2 closed.")

    if connection3.is_connected():
        cursor3.close()
        connection3.close()
        print("Connection 3 closed.")


Connection 1 closed.
Connection 2 closed.
Connection 3 closed.


Task 3

In [21]:
try:
    # Establish a connection pool [pool_name = pool_b] with two connections
    pool_b = MySQLConnectionPool(pool_name="pool_b", pool_size=2, **dbconfig)

    # Connect to execute the queries
    connection = pool_b.get_connection()
    cursor = connection.cursor()

    # Task 3: Generate the report

    # The name and EmployeeID of the Little Lemon manager
    manager_query = "SELECT Name, EmployeeID FROM Employees WHERE Role = 'Manager'"
    cursor.execute(manager_query)
    manager_result = cursor.fetchone()
    manager_name, manager_employee_id = manager_result

    print("Little Lemon manager:")
    print("Name:", manager_name)
    print("EmployeeID:", manager_employee_id)
    print()

    # The name and role of the employee who receives the highest salary
    highest_salary_query = "SELECT Name, Role FROM Employees WHERE Annual_Salary = (SELECT MAX(Annual_Salary) FROM Employees)"
    cursor.execute(highest_salary_query)
    highest_salary_result = cursor.fetchone()
    highest_salary_name, highest_salary_role = highest_salary_result

    print("Employee with the highest salary:")
    print("Name:", highest_salary_name)
    print("Role:", highest_salary_role)
    print()

    # The number of guests booked between 18:00 and 20:00
    guests_count_query = "SELECT COUNT(*) FROM Bookings WHERE BookingSlot BETWEEN '18:00' AND '20:00'"
    cursor.execute(guests_count_query)
    guests_count_result = cursor.fetchone()
    guests_count = guests_count_result[0]

    print("Number of guests booked between 18:00 and 20:00:", guests_count)
    print()

    # The full name and BookingID of all guests waiting to be seated, sorted by BookingSlot
    waiting_guests_query = "SELECT CONCAT(GuestFirstName, ' ', GuestLastName) AS Full_Name, BookingID FROM Bookings WHERE Status = 'Waiting' ORDER BY BookingSlot"
    cursor.execute(waiting_guests_query)
    waiting_guests_results = cursor.fetchall()

    print("Guests waiting to be seated:")
    for waiting_guest in waiting_guests_results:
        full_name, booking_id = waiting_guest
        print("Full Name:", full_name)
        print("BookingID:", booking_id)
        print()

except conn.Error as e:
    print("Error:", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Connection  closed.")

    


Little Lemon manager:
Name: Mario Gollini
EmployeeID: 1

Employee with the highest salary:
Name: Mario Gollini
Role: Manager

Number of guests booked between 18:00 and 20:00: 11

Error: 1054 (42S22): Unknown column 'Status' in 'where clause'
Connection  closed.


In [30]:
##creating BasicSalesReport Proc
stored_procedure = """
Create PROCEDURE BasicSalesReport()
Begin 
SELECT SUM(BillAmount) AS TotalSales,AVG(BillAmount) AS AverageSale,MIN(BillAmount) AS MinimumBillPaid,MAX(BillAmount) AS MaximumBillPaid
    FROM Orders;
END
"""
cursor.execute(stored_procedure)

In [57]:
try:
    # Establish a connection pool [pool_name = pool_b] with two connections
    pool_b = MySQLConnectionPool(pool_name="pool_b", pool_size=2, **dbconfig)


    connection = pool_b.get_connection()
    cursor = connection.cursor()
    cursor.callproc("BasicSalesReport")

    results = next(cursor.stored_results())
    data = results.fetchall()

    # Extract the values from the result
    for data in data:
         
        print("Basic Sales Report:")
        print("Total Sales:", data[0])
        print("Average Sale:", data[1])
        print("Minimum Bill Paid:", data[2])
        print("Maximum Bill Paid:", data[3])

except conn.Error as e:
    print("Error:", e)

finally:
    # Return the connection back to the pool
    if connection.is_connected():
        cursor.close()
        connection.close()


Basic Sales Report:
Total Sales: 243
Average Sale: 48.6000
Minimum Bill Paid: 37
Maximum Bill Paid: 86


Task Five

In [60]:
try:
    # Establish a connection pool [pool_name = pool_b] with two connections
    pool_b = MySQLConnectionPool(pool_name="pool_b", pool_size=2, **dbconfig)

    # Connect to execute the queries
    connection = pool_b.get_connection()
    cursor = connection.cursor(buffered=True)

    # Task 5: Retrieve and display the next three upcoming bookings
    query = """
        SELECT b.BookingSlot, CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS GuestName, CONCAT(e.Name, ' [', e.Role, ']') AS AssignedTo
        FROM Bookings AS b
        JOIN Employees AS e ON b.EmployeeID = e.EmployeeID
        ORDER BY b.BookingSlot ASC
        LIMIT 3
    """
    cursor.execute(query)

    print("Next three upcoming bookings on the kitchen screen:")
    for booking in cursor:
        booking_slot, guest_name, assigned_to = booking
        print("[{}]".format(booking_slot))
        print(guest_name)
        print("Assigned to: {}".format(assigned_to))
        print()

except conn.Error as e:
    print("Error:", e)

finally:
    # Return the connection back to the pool
    if connection.is_connected():
        cursor.close()
        connection.close()


Next three upcoming bookings on the kitchen screen:
[15:00:00]
Vanessa McCarthy
Assigned to: Giorgos Dioudis [Head Chef]

[17:30:00]
Marcos Romero
Assigned to: Fatma Kaya [Assistant Chef]

[18:00:00]
Anees Java
Assigned to: John Millar [Receptionist]



In [52]:
cursor.callproc("BasicSalesReport")
results = next(cursor.stored_results())
data = results.fetchall()
for data in data:
    print(data)

(Decimal('243'), Decimal('48.6000'), 37, 86)
